In [1]:
import pickle
from collections import OrderedDict
import pandas as pd
from pandas import DataFrame
import pickle
import sqlite3
import datetime
from dateutil.parser import parse

In [30]:
# sections = ['politics','economic','society','global','culture','digital']
idate_list = ['2018-08-16','2018-08-17','2018-08-18','2018-08-19',
              '2018-08-20','2018-08-21','2018-08-22','2018-08-23',
              '2018-08-24','2018-08-25','2018-08-26','2018-08-27']
sections = ['culture']
# idate_list = ['2018-08-23']
path_sec = 'db/inverted_index/'
path_idf = 'db/unique_nouns_idf/'

a_table = DataFrame({'rank' : range(1,11)})
a_table.set_index('rank',inplace=True)
# a_list = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.81,0.82,0.83,0.84,0.85,0.86,0.87,0.88,0.89,0.9]
a = 0.83
x_list = ['모레']

In [32]:
# for a in a_list:
kw_total=DataFrame()
for section in sections:
#     conn = sqlite3.connect('db/news_db.db')
#     cur = conn.cursor()

#     try:
#         cur.execute(f"CREATE TABLE I_value_{section} \
#                                   (index      INTEGER PRIMARY KEY, \
#                                    idate      DATE, \
#                                    term       TEXT, \
#                                    docsNum    INTEGER, \
#                                    df_sec     REAL, \
#                                    idf_all    REAL, \
#                                    i_value    REAL, \
#                                    rank_i     INTEGER, \
#                                    FOREIGN KEY (term) REFERENCES U_Term(u_term))")
#         conn.commit()
#     except:
#         pass

    def i_value_cal(a, df, idf):
        return (df * idf) / (a*df + (1-a)*idf)

    itb_idate=DataFrame()
    kw_idate=DataFrame()

    for idate in idate_list: 
        inverted = pd.read_pickle(f'{path_sec}inverted_index_{section}_{idate}.pkl')

        if parse(idate) < datetime.datetime(2018,8,23):
            idate_c = '2018-08-23'
        else:
            idate_c = idate
        idf = pd.read_pickle(f'{path_idf}unique_nouns_idf_{idate_c}_term.pkl')
        print(f'{idate} 키워드')

        term=[]
        docsNum=[]
        idf_all=[]
        itb=DataFrame()

        for t, docs in inverted.items():
            term.append(t)
            docsNum.append(len(docs))
            idf_all.append(idf[t])
        itb = DataFrame({'term':term, 'docsNum':docsNum, 'idf_all':idf_all})
        print(f'단어의 개수: {len(itb)}개')

        max_docsNum = max(itb['docsNum'])
        itb['idate'] = parse(idate)
        itb['df_sec'] = itb.docsNum / max_docsNum
        itb['i_value'] = i_value_cal(a, itb.df_sec, itb.idf_all)
        itb['rank_i'] = itb.i_value.rank(method='min', ascending=False).astype('int')

        itb.sort_values(['rank_i'], inplace=True, ascending=True)
        itb.reset_index(drop=True, inplace=True)
        itb = itb[['idate','term','docsNum','df_sec','idf_all','i_value','rank_i']]
        itb = itb.iloc[:100]
        itb[itb['term'] isin x_list]
        itb.drop([itb['term'] != '경수'])
        print(f'{"-":->50}')
        kw_idate=pd.concat([kw_idate,itb.iloc[:5,:2]])
        itb_idate = pd.concat([itb_idate,itb])
        a_temp = itb.iloc[:11,1]
        a_table = pd.concat([a_table, a_temp], axis=1, join_axes=[a_table.index])

    kw_idate['section'] = section
    kw_total = pd.concat([kw_total,kw_idate])
#     itb_idate.to_sql(f'I_value_{section}', con=conn, if_exists='append')
#     cur.execute(f'SELECT * FROM I_value_{section}').fetchall()
#     conn.commit()
# a_table

SyntaxError: invalid syntax (<ipython-input-32-2a45f4518602>, line 60)

In [29]:
a_table

,term,term,term,term,term,term,term,term,term,term,term,term
rank,,,,,,,,,,,,
1,종회,기온,날씨,남동쪽,환자,제주,솔릭,솔릭,바람,모레,비가,남부
2,불신임,아침,열대야,고기압,치료,상륙,초속,상륙,동해,기온,충청,환자
3,조계종,폭염,마음,해상,제주,솔릭,제주도,바람,비가,북부,남부,비가
4,스님,작품,고기압,세력,관통,제주도,상륙,태풍,태풍,내일,북부,곳곳
5,총무원장,사랑,서쪽,기온,솔릭,작품,태풍,강풍,사랑,비가,중부,가을
6,불교,세상,북태평양,중형,병원,마음,제주,제주,세상,마음,강원,질환
7,작품,주말,저자,북태평양,예술,바람,강풍,부근,강릉,남부,기온,건강
8,역사,사이,동쪽,솔릭,작가,태풍,작품,새벽,작품,경북,경북,문화
9,가결,날씨,폭염,대구,교수,관통,비가,북상,저자,충청,내일,충청


In [ ]:
conn = sqlite3.connect('db/news_db.db')
cur = conn.cursor()
termList = list(set(kw_total['term']))
term=[]
a_ids=[]
for t in termList:
    term.append(t)
    date = '2018-08-27'
    cur.execute(f"SELECT T.a_id, T.tfidf \
                From Term T \
                WHERE date = '{date}' AND term = '{t}'")
    result = cur.fetchall()
    a_ids.append(result)

In [ ]:
kw_a = DataFrame({'term':term,'a_ids':a_ids})

In [ ]:
with open(f'db/test.pkl','rb') as f:
    kw_a = pickle.load(f)
kw_a.loc[kw_a.term=='이해찬']

In [ ]:
with open(f'db/test.pkl','wb') as f:
    pickle.dump(kw_a, f)
    


In [ ]:
try:
    cur.execute("CREATE TABLE Kw_article(index      INTEGER PRIMARY KEY, \
                                      idate      DATE, \
                                      term       TEXT, \
                                      article    REAL, \
                                      FOREIGN KEY (term) REFERENCES U_Term(u_term))")
    conn.commit()
except:
    pass